In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

In [2]:
movies_final_2015_df = pd.read_pickle('df_all_movies_2015_pk')
movies_final_2015_df.head(10)

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015",Walt Disney,Adventure,"$742,208,942","88,043,765",https://www.the-numbers.com/movie/Star-Wars-Ep...,https://www.the-numbers.com/movie/Star-Wars-Ep...
1,2,Jurassic World,"Jun 12, 2015",Universal,Action,"$652,270,625","77,374,926",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...
2,3,Avengers: Age of Ultron,"May 1, 2015",Walt Disney,Action,"$459,005,868","54,449,094",https://www.the-numbers.com/movie/Avengers-Age...,https://www.the-numbers.com/movie/Avengers-Age...
3,4,Inside Out,"Jun 19, 2015",Walt Disney,Adventure,"$356,461,711","42,284,900",https://www.the-numbers.com/movie/Inside-Out-(...,https://www.the-numbers.com/movie/Inside-Out-(...
4,5,Furious 7,"Apr 3, 2015",Universal,Action,"$353,007,020","41,875,091",https://www.the-numbers.com/movie/Furious-7#ta...,https://www.the-numbers.com/movie/Furious-7#ta...
5,6,American Sniper,"Dec 25, 2014",Warner Bros.,Drama,"$347,897,084","41,268,930",https://www.the-numbers.com/movie/American-Sni...,https://www.the-numbers.com/movie/American-Sni...
6,7,Minions,"Jul 10, 2015",Universal,Adventure,"$336,045,770","39,863,081",https://www.the-numbers.com/movie/Minions#tab=...,https://www.the-numbers.com/movie/Minions#tab=...
7,8,The Hunger Games: Mockingja…,"Nov 20, 2015",Lionsgate,Thriller/Suspense,"$274,185,395","32,524,957",https://www.the-numbers.com/movie/Hunger-Games...,https://www.the-numbers.com/movie/Hunger-Games...
8,9,The Martian,"Oct 2, 2015",20th Century Fox,Thriller/Suspense,"$225,345,353","26,731,359",https://www.the-numbers.com/movie/Martian-The#...,https://www.the-numbers.com/movie/Martian-The#...
9,10,Cinderella,"Mar 13, 2015",Walt Disney,Drama,"$201,151,353","23,861,370",https://www.the-numbers.com/movie/Cinderella-(...,https://www.the-numbers.com/movie/Cinderella-(...


In [3]:
legs_2015_df = pd.read_pickle('legs_2015_df_pk')
legs_2015_df['Legs']

0       3.78
1       3.12
2       2.40
3       3.94
4       2.40
       ...  
807     1.00
808     1.00
809     1.00
810    84.80
811     3.70
Name: Legs, Length: 812, dtype: float64

In [4]:
movies_final_2015_df['Legs'] = legs_2015_df['Legs']

In [5]:
movies_final_2015_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs
0,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015",Walt Disney,Adventure,"$742,208,942","88,043,765",https://www.the-numbers.com/movie/Star-Wars-Ep...,https://www.the-numbers.com/movie/Star-Wars-Ep...,3.78
1,2,Jurassic World,"Jun 12, 2015",Universal,Action,"$652,270,625","77,374,926",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...,3.12
2,3,Avengers: Age of Ultron,"May 1, 2015",Walt Disney,Action,"$459,005,868","54,449,094",https://www.the-numbers.com/movie/Avengers-Age...,https://www.the-numbers.com/movie/Avengers-Age...,2.40
3,4,Inside Out,"Jun 19, 2015",Walt Disney,Adventure,"$356,461,711","42,284,900",https://www.the-numbers.com/movie/Inside-Out-(...,https://www.the-numbers.com/movie/Inside-Out-(...,3.94
4,5,Furious 7,"Apr 3, 2015",Universal,Action,"$353,007,020","41,875,091",https://www.the-numbers.com/movie/Furious-7#ta...,https://www.the-numbers.com/movie/Furious-7#ta...,2.40
...,...,...,...,...,...,...,...,...,...,...
807,808,Lou!,"Apr 24, 2015",Distrib Films,Comedy,$88,10,https://www.the-numbers.com/movie/Lou-Journal-...,https://www.the-numbers.com/movie/Lou-Journal-...,1.00
808,809,Paranoid Girls,"Nov 19, 2015",,Comedy,$54,6,https://www.the-numbers.com/movie/Chicas-Paran...,https://www.the-numbers.com/movie/Chicas-Paran...,1.00
809,810,Memphis,"Sep 5, 2014",Kino Lorber,Drama,$54,6,https://www.the-numbers.com/movie/Memphis#tab=...,https://www.the-numbers.com/movie/Memphis#tab=...,1.00
810,811,The Historian,"Nov 7, 2014",,Drama,$36,4,https://www.the-numbers.com/movie/Historian-Th...,https://www.the-numbers.com/movie/Historian-Th...,84.80


In [6]:
budget_2015_df = pd.read_pickle('budget_2015_pk')
wbo_to_budget_ratio_2015_df = pd.read_pickle('wbo_to_budget_ratio_2015_pk')

movies_final_2015_df['Budget'] = budget_2015_df['Production_Budget']
movies_final_2015_df['WBO_to_Budget_Ratio'] = (
    wbo_to_budget_ratio_2015_df['WBO to Budget Ratio'])

In [7]:
movies_final_2015_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs,Budget,WBO_to_Budget_Ratio
0,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015",Walt Disney,Adventure,"$742,208,942","88,043,765",https://www.the-numbers.com/movie/Star-Wars-Ep...,https://www.the-numbers.com/movie/Star-Wars-Ep...,3.78,306000000,6.7
1,2,Jurassic World,"Jun 12, 2015",Universal,Action,"$652,270,625","77,374,926",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...,3.12,215000000,7.8
2,3,Avengers: Age of Ultron,"May 1, 2015",Walt Disney,Action,"$459,005,868","54,449,094",https://www.the-numbers.com/movie/Avengers-Age...,https://www.the-numbers.com/movie/Avengers-Age...,2.40,365000000,3.8
3,4,Inside Out,"Jun 19, 2015",Walt Disney,Adventure,"$356,461,711","42,284,900",https://www.the-numbers.com/movie/Inside-Out-(...,https://www.the-numbers.com/movie/Inside-Out-(...,3.94,175000000,4.9
4,5,Furious 7,"Apr 3, 2015",Universal,Action,"$353,007,020","41,875,091",https://www.the-numbers.com/movie/Furious-7#ta...,https://www.the-numbers.com/movie/Furious-7#ta...,2.40,190000000,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
807,808,Lou!,"Apr 24, 2015",Distrib Films,Comedy,$88,10,https://www.the-numbers.com/movie/Lou-Journal-...,https://www.the-numbers.com/movie/Lou-Journal-...,1.00,nf,nf
808,809,Paranoid Girls,"Nov 19, 2015",,Comedy,$54,6,https://www.the-numbers.com/movie/Chicas-Paran...,https://www.the-numbers.com/movie/Chicas-Paran...,1.00,nf,nf
809,810,Memphis,"Sep 5, 2014",Kino Lorber,Drama,$54,6,https://www.the-numbers.com/movie/Memphis#tab=...,https://www.the-numbers.com/movie/Memphis#tab=...,1.00,nf,nf
810,811,The Historian,"Nov 7, 2014",,Drama,$36,4,https://www.the-numbers.com/movie/Historian-Th...,https://www.the-numbers.com/movie/Historian-Th...,84.80,nf,nf


In [8]:
theater_count_2015_df = pd.read_pickle('theater_count_2015_pk')
avg_run_2015_df = pd.read_pickle('avg_run_2015_pk')

movies_final_2015_df['Theater_Count'] = theater_count_2015_df['Theater_Count']
movies_final_2015_df['Average_Run'] = avg_run_2015_df['Average_Run']


In [9]:
source_2015_df = pd.read_pickle('source_2015_pk')
movies_final_2015_df['Source'] = source_2015_df['Source']
# movies_final_2015_df

In [10]:
creative_type_2015_df =pd.read_pickle('creative_type_2015_pk') 
movies_final_2015_df['Creative_Type'] = creative_type_2015_df['Creative_Type']
movies_final_2015_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,Source,Creative_Type
0,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015",Walt Disney,Adventure,"$742,208,942","88,043,765",https://www.the-numbers.com/movie/Star-Wars-Ep...,https://www.the-numbers.com/movie/Star-Wars-Ep...,3.78,306000000,6.7,4134,9.2,Original Screenplay,Science Fiction
1,2,Jurassic World,"Jun 12, 2015",Universal,Action,"$652,270,625","77,374,926",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...,3.12,215000000,7.8,4274,8.5,Based on Fiction Book/Short Story,Science Fiction
2,3,Avengers: Age of Ultron,"May 1, 2015",Walt Disney,Action,"$459,005,868","54,449,094",https://www.the-numbers.com/movie/Avengers-Age...,https://www.the-numbers.com/movie/Avengers-Age...,2.40,365000000,3.8,4276,7.0,Based on Comic/Graphic Novel,Super Hero
3,4,Inside Out,"Jun 19, 2015",Walt Disney,Adventure,"$356,461,711","42,284,900",https://www.the-numbers.com/movie/Inside-Out-(...,https://www.the-numbers.com/movie/Inside-Out-(...,3.94,175000000,4.9,3946,8.9,Original Screenplay,Kids Fiction
4,5,Furious 7,"Apr 3, 2015",Universal,Action,"$353,007,020","41,875,091",https://www.the-numbers.com/movie/Furious-7#ta...,https://www.the-numbers.com/movie/Furious-7#ta...,2.40,190000000,8.0,4003,6.9,Original Screenplay,Contemporary Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,808,Lou!,"Apr 24, 2015",Distrib Films,Comedy,$88,10,https://www.the-numbers.com/movie/Lou-Journal-...,https://www.the-numbers.com/movie/Lou-Journal-...,1.00,nf,nf,nf,nf,Based on Comic/Graphic Novel,Kids Fiction
808,809,Paranoid Girls,"Nov 19, 2015",,Comedy,$54,6,https://www.the-numbers.com/movie/Chicas-Paran...,https://www.the-numbers.com/movie/Chicas-Paran...,1.00,nf,nf,1,1.0,Original Screenplay,Contemporary Fiction
809,810,Memphis,"Sep 5, 2014",Kino Lorber,Drama,$54,6,https://www.the-numbers.com/movie/Memphis#tab=...,https://www.the-numbers.com/movie/Memphis#tab=...,1.00,nf,nf,nf,nf,Original Screenplay,Contemporary Fiction
810,811,The Historian,"Nov 7, 2014",,Drama,$36,4,https://www.the-numbers.com/movie/Historian-Th...,https://www.the-numbers.com/movie/Historian-Th...,84.80,nf,nf,nf,nf,Original Screenplay,Contemporary Fiction


In [11]:
movies_final_2015_df.drop(['summary_page_url', 'cast_page_url'],
                          axis = 1, inplace = True)
movies_final_2015_df.shape

(812, 14)

In [12]:
col_reorder = ['Rank', 'Movie','ReleaseDate', 'Revenues', 'Tickets_Sold',
              'Legs', 'Budget', 'WBO_to_Budget_Ratio', 'Theater_Count',
               'Average_Run', 'Genre', 'Source', 'Creative_Type', 'Distributor']

movies_final_2015_df = movies_final_2015_df[col_reorder]

In [13]:
movies_final_2015_df

,Rank,Movie,ReleaseDate,Revenues,Tickets_Sold,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,Genre,Source,Creative_Type,Distributor
0,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015","$742,208,942","88,043,765",3.78,306000000,6.7,4134,9.2,Adventure,Original Screenplay,Science Fiction,Walt Disney
1,2,Jurassic World,"Jun 12, 2015","$652,270,625","77,374,926",3.12,215000000,7.8,4274,8.5,Action,Based on Fiction Book/Short Story,Science Fiction,Universal
2,3,Avengers: Age of Ultron,"May 1, 2015","$459,005,868","54,449,094",2.40,365000000,3.8,4276,7.0,Action,Based on Comic/Graphic Novel,Super Hero,Walt Disney
3,4,Inside Out,"Jun 19, 2015","$356,461,711","42,284,900",3.94,175000000,4.9,3946,8.9,Adventure,Original Screenplay,Kids Fiction,Walt Disney
4,5,Furious 7,"Apr 3, 2015","$353,007,020","41,875,091",2.40,190000000,8.0,4003,6.9,Action,Original Screenplay,Contemporary Fiction,Universal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,808,Lou!,"Apr 24, 2015",$88,10,1.00,nf,nf,nf,nf,Comedy,Based on Comic/Graphic Novel,Kids Fiction,Distrib Films
808,809,Paranoid Girls,"Nov 19, 2015",$54,6,1.00,nf,nf,1,1.0,Comedy,Original Screenplay,Contemporary Fiction,
809,810,Memphis,"Sep 5, 2014",$54,6,1.00,nf,nf,nf,nf,Drama,Original Screenplay,Contemporary Fiction,Kino Lorber
810,811,The Historian,"Nov 7, 2014",$36,4,84.80,nf,nf,nf,nf,Drama,Original Screenplay,Contemporary Fiction,


In [14]:
cast_2015_df = pd.read_pickle('cast_2015_pk')
# cast_2015_df

In [15]:
movies_final_2015_df = movies_final_2015_df.join(cast_2015_df)

In [16]:
movies_final_2015_df.to_pickle('final_2015_pk')

In [17]:
nf_count_dict = {}

# name = 'Budget'
# movies_final_2015_df[movies_final_2015_df[name] == 'nf'][name].count()

for name in col_reorder:
    count_nf = movies_final_2015_df[movies_final_2015_df[name] == 'nf'][name].count()
    print(f'{name}   {count_nf}')


    
    

Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   537
WBO_to_Budget_Ratio   537
Theater_Count   149
Average_Run   149
Genre   0
Source   2
Creative_Type   5
Distributor   0


In [18]:
movies_final_2015_df.dtypes
# movies_final_2015_df

Rank                    object
Movie                   object
ReleaseDate             object
Revenues                object
Tickets_Sold            object
Legs                   float64
Budget                  object
WBO_to_Budget_Ratio     object
Theater_Count           object
Average_Run             object
Genre                   object
Source                  object
Creative_Type           object
Distributor             object
All_Cast_Rating        float64
Samuel L. Jackson        int64
Robert Downey, Jr.       int64
Scarlett Johansson       int64
Tom Hanks                int64
Bradley Cooper           int64
Harrison Ford            int64
Chris Evans              int64
Tom Cruise               int64
Chris Hemsworth          int64
Zoe Saldana              int64
dtype: object

In [19]:

movies_final_2015_df.replace(r'nf', np.nan, regex = True, inplace = True )


col_list = ['Revenues', 'Tickets_Sold', 'Legs', 'Budget', 'WBO_to_Budget_Ratio',
           'Theater_Count', 'Average_Run']

for col_name in col_list:
    movies_final_2015_df[col_name] = (movies_final_2015_df[col_name]
                                    .replace('[\$,]', '', regex = True))
                                        
    movies_final_2015_df[col_name] = (pd.to_numeric(movies_final_2015_df[col_name], 
                                                errors = 'coerce'))
    


In [20]:
movies_final_2015_df.dtypes
# movies_final_2015_df

Rank                    object
Movie                   object
ReleaseDate             object
Revenues                 int64
Tickets_Sold             int64
Legs                   float64
Budget                 float64
WBO_to_Budget_Ratio    float64
Theater_Count          float64
Average_Run            float64
Genre                   object
Source                  object
Creative_Type           object
Distributor             object
All_Cast_Rating        float64
Samuel L. Jackson        int64
Robert Downey, Jr.       int64
Scarlett Johansson       int64
Tom Hanks                int64
Bradley Cooper           int64
Harrison Ford            int64
Chris Evans              int64
Tom Cruise               int64
Chris Hemsworth          int64
Zoe Saldana              int64
dtype: object

In [21]:
for name in col_reorder:
    count_nf = movies_final_2015_df[movies_final_2015_df[name] == 'nf'][name].count()
    print(f'{name}   {count_nf}')

Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   0
WBO_to_Budget_Ratio   0
Theater_Count   0
Average_Run   0
Genre   0
Source   0
Creative_Type   0
Distributor   0


In [22]:
for col_name in col_reorder:
    count_nan = movies_final_2015_df[col_name].isna().sum()
    print(f'{col_name}   {count_nan}')



Rank   0
Movie   7
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   537
WBO_to_Budget_Ratio   537
Theater_Count   149
Average_Run   149
Genre   0
Source   2
Creative_Type   5
Distributor   1


In [23]:
movies_final_2015_df.to_pickle('final_2015_pk')